# Battle of the Neighborhoods - New Dispensary

# Introduction

With the legal landscape rapidly changing, regulation and decriminalization of Marijuana is significantly increasing.  A substance that was once considered dangerous through urban legends and propaganda has been debunked and is being touted for a combination of its medicinal benefits, increased safety vs alternative substances, plant by-products and more.  In the changing environment, it is the optimal time to invest in the future.  The goal of this project is to determine the optimal location for opening a new dispensary in cities in which marijuana is recreationally legal.  

# Data 

## To determine the best location for opening a number of factors will be looked at:

#### Census information collected from 
###### https://simplemaps.com/data/us-cities

#### State Laws
###### https://disa.com/map-of-marijuana-legality-by-state

#### Local Venue Data
###### https://foursquare.com

### 2017-2018 Crime Statistics  
https://ucr.fbi.gov/crime-in-the-u.s/2018/preliminary-report/tables/table-4/table-4.xls/view  

- First, the legality will be assessed for each state and city.  
- The legality will be cross referenced against a US-population database to find the most populated cities.
- Locations of current dispensaries within 1 mile (1.609 km) will be checked.  
 - If a dispensary exists, the location will be cancelled.  
- Another factor that will be assessed is the demographic of the area.  
 - Cities or neighborhoods with children and families will not be looked at in order to keep them family-friendly.  
- Since the goal of this project is to open a recreational dispensary 
 - I will be looking in areas with active nightlife and adult activities 
 - I will be looking for places with nearby fast-food, pizza restaurants and convenience stores.  Establishments are often frequented by target clientele
- I will be verifying crime rates in cities and aim for lower crime rate locations
 - Many dispensaries have been robbed since installation due to the nature of the product.  Try to minimize this risk as much as possible.


In [11]:
import numpy as np # library to handle data in a vectorized manner
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
import folium # map rendering library
from sklearn.cluster import KMeans # import k-means from clustering stage
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install beautifulsoup4 --yes

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')


Libraries imported.


In [3]:
legalized = ['Alaska','California','Nevada','Colorado', 'Oregon','Washington','Michigan','Massachusetts','Vermont','Maine','District of Columbia']

In [12]:
CLIENT_ID = 'K5HRW4OC5J4EZS14DH2OKQBNZV5GNXIOZKY03QXJ4RDCPTUD' # your Foursquare ID
CLIENT_SECRET = '4ZWZFMI0VAL01GTTVLUJSJOL0YPARUAQB225RMYILABEIQ0Q' # your Foursquare Secret
VERSION = '20190420'
limit = 50
radius = 1000
latitude = 1
longitude = 1
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentials:
CLIENT_ID: K5HRW4OC5J4EZS14DH2OKQBNZV5GNXIOZKY03QXJ4RDCPTUD
CLIENT_SECRET:4ZWZFMI0VAL01GTTVLUJSJOL0YPARUAQB225RMYILABEIQ0Q


In [94]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    limit)

In [134]:
results = requests.get(url).json()

In [135]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                  
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [136]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [138]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


## Methodology

## Results

## Discussion

## Conclusion